In [2]:

#==============================================================================
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import folium
from pyspark.sql.functions import date_format
from pyspark.sql import SparkSession, functions as F
import geopandas as gpd

#==============================================================================
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2 part 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)


/Users/Kasturi/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


22/10/08 00:39:46 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/10/08 00:39:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/08 00:39:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/08 00:39:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/08 00:39:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
%run "../scripts/outlier.py" '../scripts/paths.json'

22/10/08 00:39:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


22/10/08 00:40:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


22/10/08 00:41:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
# Number of transactions made per month

# Read the tagged model
tagged_merchants_sdf = spark.read.parquet("../data/curated/tagged_merchants.parquet")

# -----------------------------------------------------------------------------
# Join the final dataset to the tagged model
tagged_merchants_sdf = tagged_merchants_sdf.withColumnRenamed('merchant_abn',

    'tagged_merchant_abn'
)
# -----------------------------------------------------------------------------
# Calculate the BNPL earnings 
internal4.createOrReplaceTempView("join")
tagged_merchants_sdf.createOrReplaceTempView("tagged")

joint = spark.sql(""" 

SELECT *
FROM join
INNER JOIN tagged
ON join.merchant_abn = tagged.tagged_merchant_abn
""")

# -----------------------------------------------------------------------------
# Calculate the BNPL earnings 
joint = joint.drop('tagged_merchant_abn')


transactions_new_sdf = joint.withColumn("Year", 
date_format('order_datetime', 'yyyy'))

transactions_new_sdf = transactions_new_sdf.withColumn("Month", 
date_format('order_datetime', 'MMMM'))


transactions_new_sdf.createOrReplaceTempView("temp_view")

trans_2021 = spark.sql(""" 

SELECT Month, category, COUNT(merchant_abn) as transactions_2021
FROM temp_view
WHERE Year == '2021'
GROUP BY Month, category

""")

trans_2022 = spark.sql(""" 

SELECT Month, category, COUNT(merchant_abn) as transactions_2022
FROM temp_view
WHERE Year == '2022'
GROUP BY Month, category

""")


trans_2021_df = trans_2021.toPandas()
trans_2022_df = trans_2022.toPandas()

22/10/08 02:44:47 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1038700 ms exceeds timeout 120000 ms
22/10/08 02:44:47 WARN SparkContext: Killing executors is not supported by current scheduler.
